### Recommendations with MovieTweetings: Getting to Know The Data

Throughout this lesson, you will be working with the [MovieTweetings Data](https://github.com/sidooms/MovieTweetings/tree/master/recsyschallenge2014).

**Note:** There are solutions to each of the notebooks available by hitting the orange jupyter logo in the top left of this notebook.  Additionally, you can watch me work through the solutions on the screencasts that follow each workbook. 

To get started, read in the libraries and the two datasets you will be using throughout the lesson using the code below.

 

In [103]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tests as t

from datetime import datetime

%matplotlib inline


# Read in the MovieTweetings dataset originally taken from https://github.com/sidooms/MovieTweetings/tree/master/latest
movies = pd.read_csv('movies.dat', delimiter='::', header=None, names=['movie_id', 'movie', 'genre'], dtype={'movie_id': object}, engine='python')
reviews = pd.read_csv('ratings.dat', delimiter='::', header=None, names=['user_id', 'movie_id', 'rating', 'timestamp'], dtype={'movie_id': object, 'user_id': object, 'timestamp': object}, engine='python')

#### 1. Take a Look At The Data 

Take a look at the data and use your findings to fill in the dictionary below with the correct responses to show your understanding of the data.

In [3]:
movies[movies.genre.isna()]

,movie_id,movie,genre
3,25,The Oxford and Cambridge University Boat Race ...,NaN
8,0443,"Hiawatha, the Messiah of the Ojibway (1903)",NaN
32,0005530,L'héroïsme de Paddy (1915),NaN
51,009340,The Man Who Woke Up (1918),NaN
80,012844,White and Unmarried (1921),NaN
...,...,...,...
35421,11318648,Rotpunkt (2019),NaN
35432,11433098,Rob Delaney: Jackie (2020),NaN
35438,11515458,Al-Zaeem (1993),NaN
35445,11566166,Mr. Zoo: The Missing VIP (2020),NaN


In [83]:
# cell for work
movies.head()

,movie_id,movie,genre
0,0000008,Edison Kinetoscopic Record of a Sneeze (1894),Documentary|Short
1,0000010,La sortie des usines Lumière (1895),Documentary|Short
2,0000012,The Arrival of a Train (1896),Documentary|Short
3,25,The Oxford and Cambridge University Boat Race ...,NaN
4,0000091,Le manoir du diable (1896),Short|Horror


In [5]:
# cell for work
reviews.head()

,user_id,movie_id,rating,timestamp
0,1,0114508,8,1381006850
1,2,0208092,5,1586466072
2,2,0358273,9,1579057827
3,2,10039344,5,1578603053
4,2,6751668,9,1578955697


In [6]:
movies.shape

(35479, 3)

In [7]:
reviews.shape

(863866, 4)

In [36]:
# number of different movies
len(movies.movie_id.unique())

35479

In [9]:
# number of differrent ratings
reviews.shape[0]

863866

In [74]:
# number of different genres
different_genres = []
for x in movies.genre:
    try: different_genres.extend(x.split('|'))
    except: pass

different_genres = set(different_genres)
len(different_genres)

28

In [75]:
# number of unique movies
len(movies.genre.unique())

2737

In [11]:
# number of unique users
len(reviews.user_id.unique())

67353

In [12]:
# number of missing ratings
reviews[reviews.rating.isna()]

,user_id,movie_id,rating,timestamp


In [13]:
len(reviews.rating)

863866

In [14]:
# average rating across movies
reviews.rating.mean()

7.315877693994207

In [15]:
# min rating across movies
reviews.rating.min()

0

In [16]:
# max rating across movies
reviews.rating.max()

10

In [17]:
# Use your findings to match each variable to the correct statement in the dictionary


dict_sol1 = {
'The number of movies in the dataset':35479,
'The number of ratings in the dataset':863866,
'The number of different genres':2737,
'The number of unique users in the dataset':67353,
'The number missing ratings in the reviews dataset':0,
'The average rating given across all ratings':7.315877693994207,
'The minimum rating given across all ratings':0,
'The maximum rating given across all ratings':10
}

# Originally, I had this to check your solution, but the 
# links are live and updating.  That didn't end up being
# a great idea


In [18]:
reviews.isnull()

,user_id,movie_id,rating,timestamp
0,False,False,False,False
1,False,False,False,False
2,False,False,False,False
3,False,False,False,False
4,False,False,False,False
...,...,...,...,...
863861,False,False,False,False
863862,False,False,False,False
863863,False,False,False,False
863864,False,False,False,False


In [19]:
dict_sol1

{'The number of movies in the dataset': 35479,
 'The number of ratings in the dataset': 863866,
 'The number of different genres': 2737,
 'The number of unique users in the dataset': 67353,
 'The number missing ratings in the reviews dataset': 0,
 'The average rating given across all ratings': 7.315877693994207,
 'The minimum rating given across all ratings': 0,
 'The maximum rating given across all ratings': 10}

#### 2. Data Cleaning

Next, we need to pull some additional relevant information out of the existing columns. 

For each of the datasets, there are a couple of cleaning steps we need to take care of:

#### Movies
* Pull the date from the title and create new column
* Dummy the date column with 1's and 0's for each century of a movie (1800's, 1900's, and 2000's)
* Dummy column the genre with 1's and 0's

#### Reviews
* Create a date out of time stamp

You can check your results against the header of my solution by running the cell below with the **show_clean_dataframes** function.

In [112]:
movies_2 = movies.copy()

In [113]:
# create a new column date from the movie column
movies_2[['movie', 'date']] = movies_2.movie.str.split('(',expand=True)

In [114]:
movies_2.date = movies_2.date.map(lambda x: x.rstrip(')'))

In [115]:
movies_2.head()

,movie_id,movie,genre,date
0,0000008,Edison Kinetoscopic Record of a Sneeze,Documentary|Short,1894
1,0000010,La sortie des usines Lumière,Documentary|Short,1895
2,0000012,The Arrival of a Train,Documentary|Short,1896
3,25,The Oxford and Cambridge University Boat Race,NaN,1895
4,0000091,Le manoir du diable,Short|Horror,1896


In [116]:
# Dummy the date column with 1's and 0's for each century of a movie (1800's, 1900's, and 2000's)
movies_2.date[movies_2.date.str.startswith('18')]='1800'
movies_2.date[movies_2.date.str.startswith('19')]='1900'
movies_2.date[movies_2.date.str.startswith('20')]='2000'

In [117]:
movies_2.head()

,movie_id,movie,genre,date
0,0000008,Edison Kinetoscopic Record of a Sneeze,Documentary|Short,1800
1,0000010,La sortie des usines Lumière,Documentary|Short,1800
2,0000012,The Arrival of a Train,Documentary|Short,1800
3,25,The Oxford and Cambridge University Boat Race,NaN,1800
4,0000091,Le manoir du diable,Short|Horror,1800


In [118]:
# create dummy variables for each century
movies_2 = movies_2.join(pd.get_dummies(movies_2.date, dummy_na=True, prefix='d'))

In [119]:
movies_2.head()

,movie_id,movie,genre,date,d_1800,d_1900,d_2000,d_nan
0,0000008,Edison Kinetoscopic Record of a Sneeze,Documentary|Short,1800,1,0,0,0
1,0000010,La sortie des usines Lumière,Documentary|Short,1800,1,0,0,0
2,0000012,The Arrival of a Train,Documentary|Short,1800,1,0,0,0
3,25,The Oxford and Cambridge University Boat Race,NaN,1800,1,0,0,0
4,0000091,Le manoir du diable,Short|Horror,1800,1,0,0,0


In [120]:
# drop unnecessary column
movies_2.drop(columns=['date', 'd_nan'], axis=1, inplace=True)

In [121]:
movies_2.head()

,movie_id,movie,genre,d_1800,d_1900,d_2000
0,0000008,Edison Kinetoscopic Record of a Sneeze,Documentary|Short,1,0,0
1,0000010,La sortie des usines Lumière,Documentary|Short,1,0,0
2,0000012,The Arrival of a Train,Documentary|Short,1,0,0
3,25,The Oxford and Cambridge University Boat Race,NaN,1,0,0
4,0000091,Le manoir du diable,Short|Horror,1,0,0


In [124]:
# add new columns, one for each genre based on different_genres
for col in different_genres:
    movies_2[col] = 0

In [126]:
# check the new dataframe
movies_2.head()

,movie_id,movie,genre,d_1800,d_1900,d_2000,Adult,History,Reality-TV,Drama,...,War,Western,Adventure,Sport,Short,Animation,Fantasy,Game-Show,Film-Noir,Family
0,0000008,Edison Kinetoscopic Record of a Sneeze,Documentary|Short,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0000010,La sortie des usines Lumière,Documentary|Short,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0000012,The Arrival of a Train,Documentary|Short,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,25,The Oxford and Cambridge University Boat Race,NaN,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0000091,Le manoir du diable,Short|Horror,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


Look very nice! Amazing job on my side. Now populate columns with 1 if movies contains the genre

In [148]:
movies_2.columns

Index(['movie_id', 'movie', 'genre', 'd_1800', 'd_1900', 'd_2000', 'Adult',
       'History', 'Reality-TV', 'Drama', 'Biography', 'News', 'Documentary',
       'Sci-Fi', 'Action', 'Music', 'Mystery', 'Comedy', 'Crime', 'Musical',
       'Romance', 'Horror', 'Thriller', 'Talk-Show', 'War', 'Western',
       'Adventure', 'Sport', 'Short', 'Animation', 'Fantasy', 'Game-Show',
       'Film-Noir', 'Family'],
      dtype='object')

In [162]:
for i in range(movies_2.shape[0]):
    for j in different_genres:
        try: 
            if movies_2['genre'][i].find(j) != -1:
                movies_2[j][i] = 1
        except: pass

C:\Users\Andre\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


In [165]:
movies_2.head()

,movie_id,movie,genre,d_1800,d_1900,d_2000,Adult,History,Reality-TV,Drama,...,War,Western,Adventure,Sport,Short,Animation,Fantasy,Game-Show,Film-Noir,Family
0,0000008,Edison Kinetoscopic Record of a Sneeze,Documentary|Short,1,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
1,0000010,La sortie des usines Lumière,Documentary|Short,1,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
2,0000012,The Arrival of a Train,Documentary|Short,1,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
3,25,The Oxford and Cambridge University Boat Race,NaN,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0000091,Le manoir du diable,Short|Horror,1,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0


In [166]:
# drop unnecessary column
movies_2.drop('genre', axis=1, inplace=True)

In [167]:
movies_2.head()

,movie_id,movie,d_1800,d_1900,d_2000,Adult,History,Reality-TV,Drama,Biography,...,War,Western,Adventure,Sport,Short,Animation,Fantasy,Game-Show,Film-Noir,Family
0,0000008,Edison Kinetoscopic Record of a Sneeze,1,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
1,0000010,La sortie des usines Lumière,1,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
2,0000012,The Arrival of a Train,1,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
3,25,The Oxford and Cambridge University Boat Race,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0000091,Le manoir du diable,1,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0


Amazing once again

In [176]:
# create a data out of timestamp
reviews.head()

,user_id,movie_id,rating,timestamp
0,1,0114508,8,1381006850
1,2,0208092,5,1586466072
2,2,0358273,9,1579057827
3,2,10039344,5,1578603053
4,2,6751668,9,1578955697


In [186]:
ts = int(reviews['timestamp'][0])
ts

1381006850

In [192]:
print(datetime.utcfromtimestamp(ts).strftime('%Y-%m-%d %H:%M:%S'))

2013-10-05 21:00:50


In [193]:
# creaet function that does the above step, i.e. convert from ts to date
def ts_to_date(x):
    return datetime.utcfromtimestamp(x).strftime('%Y-%m-%d %H:%M:%S')

In [194]:
# check to see if it works
ts_to_date(ts)

'2013-10-05 21:00:50'

Boooooooooooooom!

In [198]:
# apply it to the new column
reviews['date'] = reviews['timestamp'].astype(int).apply(ts_to_date)

In [199]:
reviews.head()

,user_id,movie_id,rating,timestamp,date
0,1,0114508,8,1381006850,2013-10-05 21:00:50
1,2,0208092,5,1586466072,2020-04-09 21:01:12
2,2,0358273,9,1579057827,2020-01-15 03:10:27
3,2,10039344,5,1578603053,2020-01-09 20:50:53
4,2,6751668,9,1578955697,2020-01-13 22:48:17


Great performance overall

In [200]:
reviews_new, movies_new = t.show_clean_dataframes()

   Unnamed: 0  user_id  movie_id  rating   timestamp                 date  \
0           0        1     68646      10  1381620027  2013-10-12 23:20:27   
1           1        1    113277      10  1379466669  2013-09-18 01:11:09   
2           2        2    422720       8  1412178746  2014-10-01 15:52:26   
3           3        2    454876       8  1394818630  2014-03-14 17:37:10   
4           4        2    790636       7  1389963947  2014-01-17 13:05:47   

   month_1  month_2  month_3  month_4  ...  month_9  month_10  month_11  \
0        0        0        0        0  ...        0         1         0   
1        0        0        0        0  ...        0         0         0   
2        0        0        0        0  ...        0         1         0   
3        0        0        0        0  ...        0         0         0   
4        0        0        0        0  ...        0         0         0   

   month_12  year_2013  year_2014  year_2015  year_2016  year_2017  year_2018  
0     